In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import seaborn as sns
import sys
import copy
from tqdm.notebook import tqdm
from numba import jit
from scipy import stats
import networkx as nx
import random
import re
import json

plt.style.use('seaborn-deep')
plt.rcParams["text.usetex"] = True
plt.rcParams['text.latex.preamble'] = r'\usepackage{amssymb,amsmath}'

plt.rcParams["figure.figsize"] = 11.7, 8.3
plt.rcParams["figure.dpi"] = 75

plt.rcParams["font.size"] = 22
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = ["Fira Sans", 'PT Sans', 'Open Sans', 'Roboto', 'DejaVu Sans', 'Liberation Sans', 'sans-serif']

plt.rcParams["legend.frameon"] = True
plt.rcParams["legend.fancybox"] = True
plt.rcParams["legend.fontsize"] = "small"

plt.rcParams["lines.linewidth"] = 2.5
plt.rcParams["lines.markersize"] = 14
plt.rcParams["lines.markeredgewidth"] = 2

plt.rcParams["xtick.major.size"] = 8
plt.rcParams["ytick.major.size"] = 8

/tmp/ipykernel_144885/1723474923.py:19: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-deep')


### Cora

In [13]:
edges = np.loadtxt("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/new-f-mercator-datasets//cora/cora_GC.edge", dtype=str)
nodes = np.unique(edges.flatten())
nodes.shape

(2485,)

In [20]:
features = pd.DataFrame(np.loadtxt("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/new-f-mercator-datasets/cora/cora.content", dtype=str))
features.head()

,0,1,2,3,4,5,6,7,8,9,...,1425,1426,1427,1428,1429,1430,1431,1432,1433,1434
0,31336,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,Neural_Networks
1,1061127,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,Rule_Learning
2,1106406,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Reinforcement_Learning
3,13195,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Reinforcement_Learning
4,37879,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Probabilistic_Methods


In [23]:
features.columns = ['index', *[f'f{i}' for i in range(1433)], 'label']
features.drop('label', axis=1, inplace=True)
features.head()

,index,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f1423,f1424,f1425,f1426,f1427,f1428,f1429,f1430,f1431,f1432
0,31336,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,1061127,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1106406,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,13195,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,37879,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
bipartite_edges = []

for idx, row in tqdm(features.iterrows()):
    vals = row[1:].values.astype(int)
    
    tmp_features = np.nonzero(vals)[0] #- 1
    
    if str(row['index']) in nodes:
        for f in tmp_features:
            bipartite_edges.append((row['index'], f'f{f}'))

0it [00:00, ?it/s]

In [26]:
# with open("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/new-f-mercator-datasets/cora/cora_bipartite.edge", "w") as f:
#     for e in bipartite_edges:
#         f.write(f'{e[0]} {e[1]}\n')

### Facebook

In [8]:
edges = np.loadtxt("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/fb_from_roya/Facebook/Facebook.edges", dtype=str)
nodes = np.unique(edges.flatten())
nodes.shape

(12374,)

In [13]:
features = pd.DataFrame(np.loadtxt("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/fb_from_roya/Facebook/Facebook.Features", dtype=str))
features.head()

,0,1,2,3,4,5,6,7,8,9,...,3710,3711,3712,3713,3714,3715,3716,3717,3718,3719
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
features = features.reset_index()

In [15]:
features.head()

,index,0,1,2,3,4,5,6,7,8,...,3710,3711,3712,3713,3714,3715,3716,3717,3718,3719
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
bipartite_edges = []

for idx, row in tqdm(features.iterrows()):
    vals = row[1:].values.astype(int)
    
    tmp_features = np.nonzero(vals)[0] #- 1
    
    if str(row['index']) in nodes:
        for f in tmp_features:
            bipartite_edges.append((row['index'], f'f{f}'))

0it [00:00, ?it/s]

In [ ]:
# with open("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/fb_from_roya/Facebook/Facebook_bipartite.edges", "w") as f:
#     for e in bipartite_edges:
#         f.write(f'{e[0]} {e[1]}\n')

### LastFM

In [19]:
edges = np.loadtxt("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/new-f-mercator-datasets/lastfm_asia/lastfm_asia_edges.edge", dtype=str)
nodes = np.unique(edges.flatten())
nodes.shape

(7624,)

In [17]:
import json 

p = "/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/new-f-mercator-datasets/lastfm_asia/lastfm_asia_features.json"

# In total 7,842 number of features
with open(p) as json_data:
    labels = json.load(json_data)

all_features = []
for idx, feature_idx in labels.items():
    features = np.zeros(7842).astype(int)
    features[feature_idx] = 1
    all_features.append(features)
    
all_features = np.array(all_features)
all_features = pd.DataFrame(all_features)
all_features.columns = [f'f{i}' for i in range(all_features.shape[1])]
all_features['index'] = [int(x) for x in labels.keys()]

all_features

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f7833,f7834,f7835,f7836,f7837,f7838,f7839,f7840,f7841,index
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7619,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7619
7620,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7620
7621,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7621
7622,0,0,0,0,0,1,0,1,0,1,...,0,0,0,0,1,0,0,0,1,7622


In [21]:
bipartite_edges = []

for idx, row in tqdm(all_features.iterrows()):
    vals = row[:-1].values.astype(int)
    tmp_features = np.nonzero(vals)[0] #- 1
    
    if str(row['index']) in nodes:
        for f in tmp_features:
            bipartite_edges.append((row['index'], f'f{f}'))

0it [00:00, ?it/s]

In [23]:
# with open("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/new-f-mercator-datasets/lastfm_asia/lastfm_asia_bipartite.edges", "w") as f:
#     for e in bipartite_edges:
#         f.write(f'{e[0]} {e[1]}\n')

### Amazon Photo

In [24]:
edges = np.loadtxt("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/new-f-mercator-datasets/amazon_photo/amazon_photo.edge", dtype=str)
nodes = np.unique(edges.flatten())
nodes.shape

(7535,)

In [26]:
features = pd.read_csv("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/new-f-mercator-datasets/amazon_photo/amazon_photo_features.txt", sep="\s+", header=None)
features['index'] = features.index
features

,0,1,2,3,4,5,6,7,8,9,...,736,737,738,739,740,741,742,743,744,index
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,1,1,1,0,...,0,0,0,1,0,0,0,0,0,1
2,0,1,0,0,0,0,1,1,1,0,...,0,0,0,1,0,1,0,0,1,2
3,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,1,0,1,1,0,3
4,0,1,0,0,1,0,1,1,0,0,...,0,0,0,1,1,1,0,1,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7645,1,1,0,1,1,1,1,1,1,1,...,1,1,0,1,1,1,1,1,1,7645
7646,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7646
7647,0,1,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,7647
7648,1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,7648


In [27]:
bipartite_edges = []

for idx, row in tqdm(features.iterrows()):
    vals = row[:-1].values.astype(int)
    tmp_features = np.nonzero(vals)[0] #- 1
    
    if str(row['index']) in nodes:
        for f in tmp_features:
            bipartite_edges.append((row['index'], f'f{f}'))

0it [00:00, ?it/s]

In [28]:
# with open("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/new-f-mercator-datasets/amazon_photo/amazon_photo_bipartite.edge", "w") as f:
#     for e in bipartite_edges:
#         f.write(f'{e[0]} {e[1]}\n')

### IMDB

In [30]:
edges = np.loadtxt("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/new-f-mercator-datasets/imdb/imdb_mam.edge", dtype=str)
nodes = np.unique(edges.flatten())
nodes.shape

(3550,)

In [29]:
features = np.loadtxt("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/new-f-mercator-datasets/imdb/imdb_feature.txt", dtype=int)

features = pd.DataFrame(features)
features.columns = [f'f{i}' for i in features.columns]
features.reset_index(inplace=True)
features.head()

,index,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f1990,f1991,f1992,f1993,f1994,f1995,f1996,f1997,f1998,f1999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
bipartite_edges = []

for idx, row in tqdm(features.iterrows()):
    vals = row[1:].values.astype(int)
    tmp_features = np.nonzero(vals)[0] #- 1
    
    if str(row['index']) in nodes:
        for f in tmp_features:
            bipartite_edges.append((row['index'], f'f{f}'))

0it [00:00, ?it/s]

In [32]:
# with open("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/new-f-mercator-datasets/imdb/imdb_mam_bipartite.edge", "w") as f:
#     for e in bipartite_edges:
#         f.write(f'{e[0]} {e[1]}\n')

### DBLP

In [34]:
edges = np.loadtxt("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/new-f-mercator-datasets/dblp/dblp_pprefp.edge", dtype=str)
nodes = np.unique(edges.flatten())
nodes.shape

(3375,)

In [35]:
features = np.loadtxt("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/new-f-mercator-datasets/dblp/dblp_feature.txt", dtype=int)

features = pd.DataFrame(features)
features.columns = [f'f{i}' for i in features.columns]
features.reset_index(inplace=True)
features['index'] = features['index'].astype(str)
features.head()

,index,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f1990,f1991,f1992,f1993,f1994,f1995,f1996,f1997,f1998,f1999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
bipartite_edges = []

for idx, row in tqdm(features.iterrows()):
    vals = row[1:].values.astype(int)
    tmp_features = np.nonzero(vals)[0] #- 1
    
    if str(row['index']) in nodes:
        for f in tmp_features:
            bipartite_edges.append((row['index'], f'f{f}'))

0it [00:00, ?it/s]

In [37]:
# with open("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/new-f-mercator-datasets/dblp/dblp_pprefp_bipartite.edge", "w") as f:
#     for e in bipartite_edges:
#         f.write(f'{e[0]} {e[1]}\n')

### Citeseer


In [38]:
edges = np.loadtxt("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/new-f-mercator-datasets/citeseer/citeseer_GC_fixed.edge", dtype=str)
nodes = np.unique(edges.flatten())
nodes.shape

(2110,)

In [40]:
features = pd.read_csv("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/new-f-mercator-datasets/citeseer/citeseer.content", 
                     sep="\t", header=None)
features.columns = ['index', *[f'f{i}' for i in range(3703)], 'label']
features.drop('label', axis=1, inplace=True)
features.head()

/usr/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3442: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,index,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f3693,f3694,f3695,f3696,f3697,f3698,f3699,f3700,f3701,f3702
0,100157,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,100598,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,105684,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,11099,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,114091,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
bipartite_edges = []

for idx, row in tqdm(features.iterrows()):
    vals = row[1:].values.astype(int)
    tmp_features = np.nonzero(vals)[0] #- 1
    
    if str(row['index']) in nodes:
        for f in tmp_features:
            bipartite_edges.append((row['index'], f'f{f}'))

0it [00:00, ?it/s]

In [43]:
# with open("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/new-f-mercator-datasets/citeseer/citeseer_GC_fixed_bipartite.edge", "w") as f:
#     for e in bipartite_edges:
#         f.write(f'{e[0]} {e[1]}\n')

### Twitch PTBR

In [45]:
edges = np.loadtxt("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/new-f-mercator-datasets/twitch/PTBR/musae_PTBR_edges.edge", dtype=str)
nodes = np.unique(edges.flatten())
nodes.shape

(1912,)

In [47]:
p = "/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/f_mercator/twitch/PTBR/musae_PTBR_features.json"

# In total 3,169 number of features
with open(p) as json_data:
    labels = json.load(json_data)

all_features = []
for idx, feature_idx in labels.items():
    features = np.zeros(3170).astype(int)
    features[feature_idx] = 1
    all_features.append(features)
    
all_features = np.array(all_features)
all_features = pd.DataFrame(all_features)
all_features.columns = [f'f{i}' for i in range(all_features.shape[1])]
all_features['index'] = [int(x) for x in labels.keys()]
all_features

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f3161,f3162,f3163,f3164,f3165,f3166,f3167,f3168,f3169,index
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1412
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1058
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1318
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,949
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1748
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1907,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1065
1908,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1233
1909,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1483
1910,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1227


In [49]:
bipartite_edges = []

for idx, row in tqdm(all_features.iterrows()):
    vals = row[:-1].values.astype(int)
    tmp_features = np.nonzero(vals)[0] #- 1
    
    if str(row['index']) in nodes:
        for f in tmp_features:
            bipartite_edges.append((row['index'], f'f{f}'))

0it [00:00, ?it/s]

In [50]:
# with open("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/new-f-mercator-datasets/twitch/PTBR/musae_PTBR_edges_bipartite.edge", "w") as f:
#     for e in bipartite_edges:
#         f.write(f'{e[0]} {e[1]}\n')

### Cornell

In [51]:
edges = np.loadtxt("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/new-f-mercator-datasets/webkb/cornell/out1_graph_edges.edge", dtype=str)
nodes = np.unique(edges.flatten())
nodes.shape

(183,)

In [53]:
features = pd.read_csv("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/f_mercator/webkb/cornell/out1_node_feature_label.txt", 
                     sep="\t")

features = pd.concat([features['node_id'], 
                      features['label'], 
                      features['feature'].str.split(',', expand=True).astype(int)], axis=1)

features.columns = ['index', 'label', *[f'f{i}' for i in range(1703)]]

features.drop('label', axis=1, inplace=True)
features

,index,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f1693,f1694,f1695,f1696,f1697,f1698,f1699,f1700,f1701,f1702
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,3,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,178,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
179,179,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
180,180,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
181,181,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
bipartite_edges = []

for idx, row in tqdm(features.iterrows()):
    vals = row[1:].values.astype(int)
    tmp_features = np.nonzero(vals)[0] #- 1
    
    if str(row['index']) in nodes:
        for f in tmp_features:
            bipartite_edges.append((row['index'], f'f{f}'))

0it [00:00, ?it/s]

In [56]:
# with open("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/new-f-mercator-datasets/webkb/cornell/out1_graph_edges_bipartite.edge", "w") as f:
#     for e in bipartite_edges:
#         f.write(f'{e[0]} {e[1]}\n')

### Texas

In [57]:
edges = np.loadtxt("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/new-f-mercator-datasets/webkb/texas/out1_graph_edges.edge", dtype=str)
nodes = np.unique(edges.flatten())
nodes.shape

(183,)

In [58]:
features = pd.read_csv("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/f_mercator/webkb/texas/out1_node_feature_label.txt", 
                     sep="\t")

features = pd.concat([features['node_id'], 
                      features['label'], 
                      features['feature'].str.split(',', expand=True).astype(int)], axis=1)

features.columns = ['index', 'label', *[f'f{i}' for i in range(1703)]]

features.drop('label', axis=1, inplace=True)
features

,index,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f1693,f1694,f1695,f1696,f1697,f1698,f1699,f1700,f1701,f1702
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,178,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
179,179,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
180,180,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
181,181,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [59]:
bipartite_edges = []

for idx, row in tqdm(features.iterrows()):
    vals = row[1:].values.astype(int)
    tmp_features = np.nonzero(vals)[0] #- 1
    
    if str(row['index']) in nodes:
        for f in tmp_features:
            bipartite_edges.append((row['index'], f'f{f}'))

0it [00:00, ?it/s]

In [60]:
# with open("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/new-f-mercator-datasets/webkb/texas/out1_graph_edges_bipartite.edge", "w") as f:
#     for e in bipartite_edges:
#         f.write(f'{e[0]} {e[1]}\n')

### Wisconsin

In [2]:
edges = np.loadtxt("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/new-f-mercator-datasets/webkb/wisconsin/out1_graph_edges.edge", dtype=str)
nodes = np.unique(edges.flatten())
nodes.shape

(251,)

In [3]:
features = pd.read_csv("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/f_mercator/webkb/wisconsin/out1_node_feature_label.txt", 
                     sep="\t")

features = pd.concat([features['node_id'], 
                      features['label'], 
                      features['feature'].str.split(',', expand=True).astype(int)], axis=1)

features.columns = ['index', 'label', *[f'f{i}' for i in range(1703)]]

features.drop('label', axis=1, inplace=True)
features

,index,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f1693,f1694,f1695,f1696,f1697,f1698,f1699,f1700,f1701,f1702
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,246,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
247,247,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
248,248,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
249,249,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [4]:
bipartite_edges = []

for idx, row in tqdm(features.iterrows()):
    vals = row[1:].values.astype(int)
    tmp_features = np.nonzero(vals)[0] #- 1
    
    if str(row['index']) in nodes:
        for f in tmp_features:
            bipartite_edges.append((row['index'], f'f{f}'))

0it [00:00, ?it/s]

In [15]:
(features[features['index'] == 161].drop('index', axis=1).values > 0).sum()

296

In [64]:
# with open("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/new-f-mercator-datasets/webkb/wisconsin/out1_graph_edges_bipartite.edge", "w") as f:
#     for e in bipartite_edges:
#         f.write(f'{e[0]} {e[1]}\n')

### Wikipedia - Chameleon

In [2]:
edges = np.loadtxt("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/new-f-mercator-datasets/wikipedia/wikipedia/chameleon/musae_chameleon_edges.edge", dtype=str)
nodes = np.unique(edges.flatten())
nodes.shape

(2277,)

In [5]:
p = "/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/new-f-mercator-datasets/wikipedia/wikipedia/chameleon/musae_chameleon_features.json"

with open(p) as json_data:
    labels = json.load(json_data)

all_features = []
for idx, feature_idx in labels.items():
    features = np.zeros(3132).astype(int)
    features[feature_idx] = 1
    all_features.append(features)
    
all_features = np.array(all_features)
all_features = pd.DataFrame(all_features)
all_features.columns = [f'f{i}' for i in range(all_features.shape[1])]
all_features['index'] = [int(x) for x in labels.keys()]
all_features

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f3123,f3124,f3125,f3126,f3127,f3128,f3129,f3130,f3131,index
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2098
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,47
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1674
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1207
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2272,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,846
2273,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,90
2274,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2141
2275,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,951


In [6]:
bipartite_edges = []

for idx, row in tqdm(all_features.iterrows()):
    vals = row[:-1].values.astype(int)
    tmp_features = np.nonzero(vals)[0] #- 1
    
    if str(row['index']) in nodes:
        for f in tmp_features:
            bipartite_edges.append((row['index'], f'f{f}'))

0it [00:00, ?it/s]

In [7]:
# with open("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/new-f-mercator-datasets/wikipedia/wikipedia/chameleon/musae_chameleon_edges_bipartite.edge", "w") as f:
#     for e in bipartite_edges:
#         f.write(f'{e[0]} {e[1]}\n')

### Wikipedia -- Crocodiles

In [3]:
edges = np.loadtxt("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/new-f-mercator-datasets/wikipedia/wikipedia/crocodile/musae_crocodile_edges.edge", dtype=str)
nodes = np.unique(edges.flatten())
nodes.shape

(11631,)

In [5]:
p = "/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/new-f-mercator-datasets/wikipedia/wikipedia/crocodile/musae_crocodile_features.json"

with open(p) as json_data:
    labels = json.load(json_data)

all_features = []
for idx, feature_idx in labels.items():
    features = np.zeros(13183).astype(int)
    features[feature_idx] = 1
    all_features.append(features)
    
all_features = np.array(all_features)
all_features = pd.DataFrame(all_features)
all_features.columns = [f'f{i}' for i in range(all_features.shape[1])]
all_features['index'] = [int(x) for x in labels.keys()]
all_features

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f13174,f13175,f13176,f13177,f13178,f13179,f13180,f13181,f13182,index
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8927
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3486
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8534
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2988
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11626,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6948
11627,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9594
11628,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6053
11629,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,496


In [6]:
bipartite_edges = []

for idx, row in tqdm(all_features.iterrows()):
    vals = row[:-1].values.astype(int)
    tmp_features = np.nonzero(vals)[0] #- 1
    
    if str(row['index']) in nodes:
        for f in tmp_features:
            bipartite_edges.append((row['index'], f'f{f}'))

0it [00:00, ?it/s]

In [7]:
# with open("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/new-f-mercator-datasets/wikipedia/wikipedia/crocodile/musae_crocodile_edges_bipartite.edge", "w") as f:
#     for e in bipartite_edges:
#         f.write(f'{e[0]} {e[1]}\n')

### Wikipedia - Squirrels

In [8]:
edges = np.loadtxt("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/new-f-mercator-datasets/wikipedia/wikipedia/squirrel/musae_squirrel_edges.edge", dtype=str)
nodes = np.unique(edges.flatten())
nodes.shape

(5201,)

In [9]:
p = "/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/new-f-mercator-datasets/wikipedia/wikipedia/squirrel/musae_squirrel_features.json"

with open(p) as json_data:
    labels = json.load(json_data)

all_features = []
for idx, feature_idx in labels.items():
    features = np.zeros(3148).astype(int)
    features[feature_idx] = 1
    all_features.append(features)
    
all_features = np.array(all_features)
all_features = pd.DataFrame(all_features)
all_features.columns = [f'f{i}' for i in range(all_features.shape[1])]
all_features['index'] = [int(x) for x in labels.keys()]
all_features

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f3139,f3140,f3141,f3142,f3143,f3144,f3145,f3146,f3147,index
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1207
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2294
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3315
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,196
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5196,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2640
5197,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,496
5198,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,873
5199,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2479


In [10]:
bipartite_edges = []

for idx, row in tqdm(all_features.iterrows()):
    vals = row[:-1].values.astype(int)
    tmp_features = np.nonzero(vals)[0] #- 1
    
    if str(row['index']) in nodes:
        for f in tmp_features:
            bipartite_edges.append((row['index'], f'f{f}'))

0it [00:00, ?it/s]

In [12]:
# with open("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/new-f-mercator-datasets/wikipedia/wikipedia/squirrel/musae_squirrel_edges_bipartite.edge", "w") as f:
#     for e in bipartite_edges:
#         f.write(f'{e[0]} {e[1]}\n')

### Film

In [30]:
edges = np.loadtxt("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/new-f-mercator-datasets/film/out1_graph_edges.edge", dtype=str)
nodes = np.unique(edges.flatten())
nodes.shape

(7600,)

In [27]:
features = pd.read_csv("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/new-f-mercator-datasets/film/out1_node_feature_label.txt", sep="\t")
features.columns = ['index', 'feature', 'label']
indices = [[int(y) for y in x.split(',')] for x in features['feature'].values]
feature_matrix = np.zeros((len(indices), 932), dtype=int)
for i in range(len(indices)):
    feature_matrix[i, indices[i]] = 1

features = pd.concat([features['index'], features['label'], pd.DataFrame(feature_matrix)], axis=1)
features.columns = ['index', 'class', *[f'f{i}' for i in range(932)]]
features

,index,class,f0,f1,f2,f3,f4,f5,f6,f7,...,f922,f923,f924,f925,f926,f927,f928,f929,f930,f931
0,4873,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1216,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1063,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3118,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2321,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7595,2956,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7596,1224,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7597,2684,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7598,4201,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
bipartite_edges = []

for idx, row in tqdm(features.iterrows()):
    vals = row[2:].values.astype(int)
    tmp_features = np.nonzero(vals)[0] #- 1
    
    if str(row['index']) in nodes:
        for f in tmp_features:
            bipartite_edges.append((row['index'], f'f{f}'))

0it [00:00, ?it/s]

In [33]:
# with open("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/new-f-mercator-datasets/film/film_bipartite.edge", "w") as f:
#     for e in bipartite_edges:
#         f.write(f'{e[0]} {e[1]}\n')